<a href="https://colab.research.google.com/github/isdeniz/issue_labels/blob/main/max_seq_200_roberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install simpletransformers==0.64.3

In [2]:
import pandas as pd
from functools import partial
import sklearn
import torch
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import logging
import datetime
pd.set_option('display.max_colwidth', None)
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)
torch.multiprocessing.set_sharing_strategy('file_system')
cuda_available = torch.cuda.is_available()
print("Is cuda available?", cuda_available)

Is cuda available? True


In [3]:
import torch.multiprocessing
torch.multiprocessing.set_start_method('spawn', force = True)

In [4]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [5]:
train = pd.read_csv('data/train_clean_concat_500.csv')
test = pd.read_csv('data/test_clean_concat_500.csv')
print('Number of issues: ', train.shape, test.shape)

Number of issues:  (171392, 2) (19044, 2)


In [6]:
train.head()

,text,labels
0,2021-01-13T19:22:34Z NONE OpenAPITools/openapi-generatorreq update of the spring dependencies and evaluation of all the options B is your feature request related to a problem please describethe problem is the spring dependencies are using old libraries or libraries that are not relevant anymore or are superseeded by better alternatives springfox for example could be replaced by springdoc also now it only generates maven projects there should also be an option to generate only sourcecode wihtout the project files this would be a breaking change for this generator also the integration of some plugins that do some static analyzing per pipeline run on the generated projects describe the solution youd likei want to update the templates for spring so that we can build gradlemaven sources with uptodate libraries describe alternatives youve consideredone could set the dependencies in a parent object and leave the generated project as is but this comes with the trouble of defining old libs in the own sourcecode additional contexti would completely implement this im also open for suggestions just give me a hint if this project would welcome this update,1
1,2017-09-06T21:07:34Z NONE SeleniumHQ/seleniumsomehow provide a way to stop the selenium standalone server programmatically B meta os allselenium version 353browser all expected behavior previously there was a way to shut down selenium server using httplocalhost64146seleniumserverdrivercmdshutdownseleniumserver actual behavior no documented way to shut down selenium standalone server steps to reproduce start selenium servernote no way to stop server with an api request,3
2,2018-11-11T20:39:28Z CONTRIBUTOR bazelbuild/bazelbazel crashes on windows with compilermsysgcc B exception stack tracecaused by javalangillegalstateexception bazelwindows_unix_root jvm flag is not set use the host_jvm_args flag for example host_jvm_argsdbazelwindows_unix_rootctoolsmsys64 at comgoogledevtoolsbuildlibutildependencysetwindowspathgetunixrootdependencysetjava285 at comgoogledevtoolsbuildlibutildependencysetwindowspathtranslatewindowspathdependencysetjava269 at comgoogledevtoolsbuildlibutildependencysetwindowspathaccess000dependencysetjava254 at comgoogledevtoolsbuildlibutildependencysettranslatepathdependencysetjava112 at comgoogledevtoolsbuildlibutildependencysetadddependencydependencysetjava103 at comgoogledevtoolsbuildlibutildependencysetprocessdependencysetjava154 at comgoogledevtoolsbuildlibutildependencysetreaddependencysetjava121 at comgoogledevtoolsbuildlibrulescppcppcompileactionprocessdepsetcppcompileactionjava1277 at comgoogledevtoolsbuildlibrulescppcppcompileactiondiscoverinputsfromdotdfilescppcompileactionjava1249 at comgoogledevtoolsbuildlibrulescppcppcompileactionexecutecppcompileactionjava1188 at comgoogledevtoolsbuildlibskyframeskyframeactionexecutorexecuteactiontaskskyframeactionexecutorjava941 at comgoogledevtoolsbuildlibskyframeskyframeactionexecutorpreparescheduleexecuteandcompleteactionskyframeactionexecutorjava872 at comgoogledevtoolsbuildlibskyframeskyframeactionexecutoraccess900skyframeactionexecutorjava114 at comgoogledevtoolsbuildlibskyframeskyframeactionexecutoractionrunnercallskyframeactionexecutorjava731 at comgoogledevtoolsbuildlibskyframeskyframeactionexecutoractionrunnercallskyframeactionexecutorjava685 at javautilconcurrentfuturetaskrunfuturetaskjava266 at comgoogledevtoolsbuildlibskyframeskyframeactionexecutorexecuteactionskyframeactionexecutorjava426 at comgoogledevtoolsbuildlibskyframeactionexecutionfunctioncheckcacheandexecuteifneededactionexecutionfunctionjava490 at comgoogledevtoolsbuildlibskyframeactionexecutionfunctioncomputeactionexecutionfunctionjava208 at comgoogledevtoolsbuildskyframeabstractparallelevaluatorevaluaterunabstractparallelevaluatorjava363 4 more,0
3,2013-06-24T10:17:01Z COLLABORATOR spring-projects/spring-frameworkcontentnegotiatingviewresolver does not select any views if no content types are requested spr10683 B jeff knechthttpsji

In [7]:
test.head()

,text,labels
0,2015-06-04T15:59:27Z CONTRIBUTOR zaproxy/zaproxy cookie without secure flag alerts not showing with spider B the alerts of the passive scanner cookie without secure flag are not shown when thepages are visited with the spiderthis only happens when the site is accessed by the spider through http if its accessedthrough https the alerts will be shownoriginal issue reported on codegooglecom by thc202 on 20120109 153837,0
1,2022-07-06T18:33:27Z NONE elastic/elasticsearch custom rescorer that returns a topdocs with zero scoredocs causes arrayindexoutofboundsexception within rescorephase B i have a custom rescorer that filters out documents below a certain threshold if all documents within the rescorer have a score below the threshold an empty topdocs is returned when an empty topdocs is returned rescorephase throws an arrayindexoutofboundsexception as the code directly references topdocsscoredocs0score without checking the size of the topdocsempty topdocs new topdocsnew new totalhits0 totalhitsrelationequal_to new scoredocs0the bug appears to have been first introduced in v700alpha1 and has been confirmed to appear on master to this day the commit responsible can be found in the upgrade to a lucene 8 snapshothttpsgithubcomelasticelasticsearchcommit7ad71f906a9608e1c168358624f4b0a39b240296 commit pr issue in the linked commit you can see that the method signature for function topdocs changed which inadvertently requires the topdocs to not be empty at this stagea fix for this bug could be to check the size of the topdocs returned after running through all rescorers,1
2,2021-09-22T13:22:58Z NONE TeamNewPipe/NewPipe suggest new videos to watch B if you dont fill in the template properly your issue is liable to be closed if you feel tiredlazy right now open your issue some other time well wait the comments between these brackets wont show up in the submitted issue as you can see in the preview checklist this checklist is compulsory the first box has been checked for you to show you how it is done x i checked but didnt find any duplicates open or closed of this issue in the repo seriously check o_o x i have read the contribution guidelines given at httpsgithubcomteamnewpipenewpipeblobheadgithubcontributingmd x this issue contains only one feature request i will open one issue for every feature i want to request describe the feature you wantid like to see a new screen being added that suggests you videos to watch id like to be able to select a few of my favorite channels and then want to be presented with some older videos or videos i havent watched in some time basically a random way of rewatching old videos this way there can be suggestions without the need for data collection either random videos suggestions or old videos from channels youve watched the most based on watch history or select your favorite channels and get presented old videos you havent seen in a while optionally also describe alternatives youve consideredexample z is also a good alternative not as good as y but at least or i considered z but that didnt turn out to be a good idea because is your feature request related to a problem please describe it a clear and concise description of what the problem is maybe the developers and the community could brainstorm and come up with a better solution to your problem if they exist link to related issues andor prs for developers to keep track easierexample i want to do x but there is no way to do it additional context add any other context like screenshots about the feature request hereexample heres a photo of my cat how will youeveryone benefit from this featurethis feature will make newpipe more interesting to use when you have finished watching your chronological feedsometimes you just want to continue watching some more videos in other apps this would be achieved by taking in your watch history and suggestions new content based on thatnewpipe obviously doesnt do that my suggested,1
3,2018-08-18T10:30:03Z NONE fa

In [8]:
lr = 3e-5
drp = 0
epochs = 4
batch_t = 100
batch_e = 100
# max_seq = 200
max_seq = 200
name = 'roberta'
ver = 'roberta-base'
output_name = 'outputs/' + name

def create_model(name, ver, lr, drp, epochs, batch_t, batch_e, max_seq):
    model_args = ClassificationArgs()
    model_name = name
    model_version = ver
    model_args.learning_rate = lr
    model_args.num_train_epochs = epochs
    model_args.eval_batch_size = batch_t
    model_args.train_batch_size = batch_e
    model_args.max_seq_length = max_seq
    model_args.n_gpu = 2
    # model_args.n_gpu = 1
    model_args.output_dir = output_name +'/'
    model_args.overwrite_output_dir = True
    model_args.reprocess_input_data = True
    model_args.preprocess_inputs = True
    model_args.save_steps = -1
    model_args.save_model_every_epoch = False

    model = ClassificationModel(model_name, model_version, args = model_args,
                                num_labels = 4,
                                use_cuda = cuda_available)

    return model

In [9]:
def calc(p1, p2, func, **kwargs):
    return func(p1, p2, **kwargs)

metrics_recom = {
    "accuracy": partial(calc,func=sklearn.metrics.accuracy_score) ,
    "p_micro": partial(calc,func=sklearn.metrics.precision_score,average='micro'),
    "p_macro": partial(calc,func=sklearn.metrics.precision_score,average='macro'),
    "p_w": partial(calc,func=sklearn.metrics.precision_score,average='weighted'),
    "r_micro": partial(calc,func=sklearn.metrics.recall_score,average='micro'),
    "r_macro": partial(calc,func=sklearn.metrics.recall_score,average='macro'),
    "r_w": partial(calc,func=sklearn.metrics.recall_score,average='weighted'),
    "f_micro": partial(calc,func=sklearn.metrics.f1_score,average='micro'),
    "f_macro": partial(calc,func=sklearn.metrics.f1_score,average='macro'),
    "f_w": partial(calc,func=sklearn.metrics.f1_score,average='weighted'),
    "classificationReport": partial(calc,func=sklearn.metrics.classification_report, output_dict = True)
}

In [10]:
model = create_model(name, ver, lr, drp, epochs, batch_t, batch_e, max_seq)
start = datetime.datetime.now()
print('-'*5,  name, ', start time:',
datetime.datetime.strftime(datetime.datetime.today(), '%d/%m/%Y-%H:%M'), '-'*5)
model.train_model(train_df = train, **metrics_recom)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'clas

----- roberta , start time: 08/05/2024-19:40 -----


  0%|          | 0/171392 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/1714 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/1714 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/1714 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/1714 [00:00<?, ?it/s]

(6856, 0.5706216169732037)

In [11]:
hours, remainder = divmod((datetime.datetime.now() - start).total_seconds(), 3600)
minutes, seconds = divmod(remainder, 60)
print('-'*5,  name, ', duration is:', '%dh:%dm:%ds' % (hours, minutes, seconds), '-'*5, '\n\n')
results, model_outputs, wrong_pred = model.eval_model(test, verbose=True, **metrics_recom)
results

----- roberta , duration is: 2h:0m:50s ----- 




  0%|          | 0/19044 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/191 [00:00<?, ?it/s]

{'mcc': 0.7060255275933421,
 'accuracy': 0.7925330812854442,
 'p_micro': 0.7925330812854442,
 'p_macro': 0.764239968763466,
 'p_w': 0.7888089417038358,
 'r_micro': 0.7925330812854442,
 'r_macro': 0.7433258000890585,
 'r_w': 0.7925330812854442,
 'f_micro': 0.7925330812854442,
 'f_macro': 0.7496360311396306,
 'f_w': 0.7883253994685459,
 'classificationReport': {'0.0': {'precision': 0.784582012347739,
   'recall': 0.8487364620938628,
   'f1-score': 0.8153992889967918,
   'support': 5540},
  '1.0': {'precision': 0.7477840451248993,
   'recall': 0.7793952967525196,
   'f1-score': 0.763262508567512,
   'support': 3572},
  '2.0': {'precision': 0.8476882430647292,
   'recall': 0.8552578968412635,
   'f1-score': 0.8514562462681616,
   'support': 7503},
  '3.0': {'precision': 0.676905574516496,
   'recall': 0.4899135446685879,
   'f1-score': 0.5684260807260568,
   'support': 2429},
  'accuracy': 0.7925330812854442,
  'macro avg': {'precision': 0.764239968763466,
   'recall': 0.7433258000890585,
